This notebook is used to create the index arrays for our edges after spatial reduction to fit our model in stan in R.

In [1]:
import xarray as xr
import numpy as np

In [2]:
tp_quantile = 0.0013141632
ws10_quantile = 11.48831558227539

Let us analyse hurricane Michael for now.

In [3]:
speeds = xr.open_dataset("../data_grib/MICHAEL_2018_10.grib", engine='cfgrib', backend_kwargs={'filter_by_keys': {'paramId': [165, 166]}})
rainfall = xr.open_dataset("../data_grib/MICHAEL_2018_10.grib", engine='cfgrib', backend_kwargs={'filter_by_keys': {'paramId': [228]}})

speeds = speeds.to_dataframe()
rainfall = rainfall.to_dataframe()

u_speed = speeds.u10
v_speed = speeds.v10
ws10 = np.sqrt(u_speed**2 + v_speed**2)

speeds["ws10"] = ws10

mask_ws10 = speeds['ws10'] > ws10_quantile
mask_tp = rainfall['tp'] > tp_quantile

temp = mask_ws10.groupby(["latitude", "longitude"]).sum()
mask_ws10_average = temp > 0

temp = mask_tp.groupby(["latitude", "longitude"]).sum()
mask_tp_average = temp > 0

combined_mask = mask_ws10_average & mask_tp_average

Ignoring index file '../data_grib/MICHAEL_2018_10.grib.5b7b6.idx' incompatible with GRIB file
Ignoring index file '../data_grib/MICHAEL_2018_10.grib.5b7b6.idx' incompatible with GRIB file


In [4]:
mask = combined_mask.unstack()
lat = mask.index.values
lon = mask.columns.values

In [5]:
# Initialise our adjacency matrix which would be NxN

N = len(lat)*len(lon)
adj = np.zeros((N, N))

adj

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
# Define a function to check if node i is kept after spatial reduction

lat_temp = lat[::-1]

def retained(mask, i):

    lat_i = i % len(lat)
    lon_i = i // len(lat)

    return mask.loc[lat_temp[lat_i], lon[lon_i]]

In [55]:
# Start filling out the matrix where Aij = 1 if i and j are neighbours
# Start from top left of the map i.e lon = 95, lat = 32 and go down

i,j = 0,0

# Fill in the upper triangular of the matrix

for i in range(0,N):

    #Check if node i is retained

    if retained(mask, i):

        for j in range(i+1,N):
            
            # Check if node j is retained
                
            if retained(mask, j):

                # Check if nodes i and j are neigbours

                lat_i, lon_i = i % len(lat), i // len(lat)
                lat_j, lon_j = j % len(lat), j // len(lat)

                if lat_i == lat_j and abs(lon_j - lon_i) == 1:

                    adj[i,j] = 1

                elif lon_i == lon_j and abs(lat_i - lat_j) == 1:

                    adj[i,j] = 1


In [56]:
# Filling in the lower triangular
adj_complete = adj + adj.T

Now I want to convert the Adjacency Matrix into two lists defining all our edges.

In [58]:
N_edges = int(adj.sum())
N_edges

3515

In [59]:
node1 = np.zeros(N_edges)
node2 = np.zeros(N_edges)

In [65]:
# This is the number of neighbours each node has, we are using the upper triangular to avoid a double count
# i.e if nodes 3 and 9 are neigbours, we will only consider 3 --> 9 and not 9 --> 3.
neighbours = adj.sum(axis=1)
neighbours

array([0., 0., 0., ..., 0., 0., 0.], shape=(4141,))

In [82]:
i = 0

for k in range(len(neighbours)):

    count = int(neighbours[k])
    
    if count != 0:

        locations = np.nonzero(adj[k])

        for m in range(count):

            # node1 stores the start index of the edges
            node1[i] = k+1

            # node2 stores the end index of the edges
            node2[i] = locations[0][m] + 1

            i += 1 

In [86]:
node1

array([ 835.,  875.,  875., ..., 4128., 4129., 4130.], shape=(3515,))

In [84]:
node2

array([ 876.,  876.,  916., ..., 4129., 4130., 4131.], shape=(3515,))

Now I want to export these lists so I can use them in R

In [87]:
np.savetxt("node1.csv", node1, delimiter=",", fmt="%d")

np.savetxt("node2.csv", node2, delimiter=",", fmt="%d")

In [16]:
y = speeds.ws10.loc["2018-10-06 00:00:00"].unstack()

In [20]:
lst = y.to_numpy().flatten(order='F').tolist()

In [27]:
np.savetxt("y.csv", lst, delimiter=",", fmt="%.6f")